## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 17:51:56


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)  

## Objective function

In [11]:
def _FOag(m):
    return sum([m.EEVmax[ev] - m.EEV[ev,t] + m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])  
model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpud5yremw
  Lower bound: 441610867.4512243
  Upper bound: 441610867.4512243
  Number of objectives: 1
  Number of constraints: 397151
  Number of variables: 338401
  Number of nonzeros: 735551
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 1.38
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 4.4161086745e+08
  Error rc: 0
  Time: 3.730412006378174
# ----------------------------------------------------------
#   Solution Information
# ---------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

441610867.45122325

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 17:54:03
Dif: 0:02:07


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
PEVdc_df = ext_pyomo_vals(model.PEVdc)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 4048543.068894738


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 69529409.54931077


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,61206.650,61206.650,61206.650,61206.650,61206.650,61206.650,61206.650,61206.650,61206.650,61206.650,...,68000.000,68000.000,68000.00,68000.000,68000.000,68000.0,68000.000,68000.00,68000.000,68000.0
2,60420.000,63840.000,67260.000,70680.000,74100.000,77520.000,80940.000,84360.000,87780.000,91200.000,...,95000.000,94344.125,93688.25,93032.375,93032.375,92376.5,91720.625,91064.75,93828.875,95000.0
3,61126.050,61126.050,61126.050,61126.050,61126.050,61126.050,61126.050,61126.050,61126.050,61126.050,...,68000.000,68000.000,68000.00,68000.000,68000.000,68000.0,68000.000,68000.00,68000.000,68000.0
4,63840.000,70680.000,77520.000,84360.000,91200.000,95000.000,95000.000,95000.000,95000.000,95000.000,...,93923.000,93923.000,93923.00,93923.000,93923.000,93923.0,93923.000,93923.00,93923.000,95000.0
5,11515.500,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,...,13500.000,13500.000,13500.00,13500.000,13500.000,13500.0,13500.000,13500.00,13500.000,13500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,63024.245,63024.245,63024.245,63024.245,63024.245,63024.245,63024.245,63024.245,63024.245,63024.245,...,71000.000,71000.000,71000.00,71000.000,71000.000,71000.0,71000.000,71000.00,71000.000,71000.0
2347,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,...,13500.000,13500.000,13500.00,13500.000,13500.000,13500.0,13500.000,13500.00,13500.000,13500.0
2348,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,13500.000,...,13500.000,13500.000,13500.00,13500.000,13500.000,13500.0,13500.000,13500.00,13500.000,13500.0
2349,64680.000,64680.000,64680.000,64680.000,64680.000,64680.000,64680.000,64680.000,64680.000,64680.000,...,105234.445,107800.000,107800.00,107800.000,107800.000,107800.0,107800.000,107800.00,107800.000,107800.0


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,22000.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,7670.210526,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,3600.000000,3600.000000,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.0,3600.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3600.0,1232.763158
3,22000.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,7839.894737,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,7200.000000,7200.000000,7200.0,7200.0,7200.0,4000.0,0.0,0.0,0.0,1133.684211,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2267.368421
5,3600.000000,2088.947368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,4.736842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,22000.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,8896.326316,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2347,5687.052632,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.842105,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2348,5684.210526,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,37.421053,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.421053
2349,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,22000.000000,2700.584211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1311.110526


In [20]:
PEV_df.sum()

1     1.525093e+07
2     4.850381e+06
3     4.135748e+06
4     3.905874e+06
5     2.892210e+06
6     1.851446e+06
7     1.413739e+06
8     1.279811e+06
9     1.080418e+06
10    6.793493e+05
11    4.746029e+05
12    6.422178e+04
13    1.528077e+05
14    1.616257e+07
15    1.200197e+07
16    3.630860e+05
17    0.000000e+00
18    0.000000e+00
19    6.659760e+04
20    1.547177e+05
21    6.855315e+04
22    1.547177e+05
23    1.168652e+06
24    1.357010e+06
dtype: float64

In [21]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [22]:
Etriprelax_df.sum()

1        0.00
2        0.00
3        0.00
4        0.00
5        0.00
6        0.00
7        0.00
8        0.00
9        0.00
10       0.00
11       0.00
12       0.00
13       0.00
14       0.00
15       0.00
16       0.00
17       0.00
18       0.00
19       0.00
20       0.00
21       0.00
22       0.00
23       0.00
24    1829.14
dtype: float64

In [23]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [24]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [25]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [26]:
Etripn_df.sum()

1     2.612024e+05
2     6.760249e+03
3     2.264454e+03
4     0.000000e+00
5     2.015204e+03
6     0.000000e+00
7     2.236121e+03
8     1.784238e+03
9     0.000000e+00
10    1.718482e+05
11    3.989772e+05
12    0.000000e+00
13    1.929628e+05
14    8.230019e+05
15    3.673955e+05
16    3.712070e+05
17    1.048677e+05
18    2.242252e+05
19    7.264225e+04
20    2.518741e+05
21    2.880438e+05
22    2.639219e+05
23    2.856192e+05
24    1.036363e+06
dtype: float64

## Save results in csv files

In [27]:
import os 
folder = 'results_Dumb_V2G' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')